In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import tqdm
%matplotlib inline

In [3]:
PATH_DATA = './PhonePi/data/'
DIR = [os.path.join(PATH_DATA, o) for o in os.listdir(PATH_DATA) 
                    if os.path.isdir(os.path.join(PATH_DATA,o))]

In [4]:
DIR

['./PhonePi/data/camminata-alberto',
 './PhonePi/data/camminata-alberto.2',
 './PhonePi/data/camminata-anna',
 './PhonePi/data/camminata-daniele',
 './PhonePi/data/camminata-emanuele',
 './PhonePi/data/camminata-luisa',
 './PhonePi/data/camminata-luisa.2',
 './PhonePi/data/camminata-simone',
 './PhonePi/data/camminata_tasca-alberto',
 './PhonePi/data/camminata_tasca-alberto.2',
 './PhonePi/data/camminata_tasca-daniele',
 './PhonePi/data/camminata_tasca-emanuele',
 './PhonePi/data/camminata_tasca-luisa',
 './PhonePi/data/camminata_tasca-luisa.2',
 './PhonePi/data/camminata_tasca-simone',
 './PhonePi/data/corsa-alberto',
 './PhonePi/data/corsa-anna',
 './PhonePi/data/corsa-luisa',
 './PhonePi/data/corsa_tasca-alberto',
 './PhonePi/data/corsa_tasca-luisa',
 './PhonePi/data/quotidiano-alberto',
 './PhonePi/data/quotidiano-luisa',
 './PhonePi/data/salti-alberto',
 './PhonePi/data/shake-alberto',
 './PhonePi/data/shake-anna',
 './PhonePi/data/shake-daniele',
 './PhonePi/data/shake-luisa',
 '

plt.figure(figsize=(13,4*len(DIR)))
nrow = len(DIR)
for i in tqdm.tqdm(range(len(DIR))):
    data = pd.read_csv(DIR[i] + "/accelerometer.txt", names = ["user", "type", "t", "ax", "ay", "az"])
    data["t"] = data["t"] - data["t"].iloc[0]
    data[(data["t"] > 7000) & (data["t"] < (data["t"].max()-7000))]
    data["a"] = (pd.to_numeric(data["ax"])**2 + pd.to_numeric(data["ay"])**2 + pd.to_numeric(data["az"])**2)**0.5 # accelerazione in modulo
    ax = plt.subplot(nrow, 1, i+1, ylim=(0,120))
    ax.set_title(DIR[i].split("/")[-1])
    plt.plot(data["t"],data["a"])
    
plt.show()

In [5]:
tipo=[(dir.split("/")[-1]).split(".")[0] for dir in DIR]
tipo

['camminata-alberto',
 'camminata-alberto',
 'camminata-anna',
 'camminata-daniele',
 'camminata-emanuele',
 'camminata-luisa',
 'camminata-luisa',
 'camminata-simone',
 'camminata_tasca-alberto',
 'camminata_tasca-alberto',
 'camminata_tasca-daniele',
 'camminata_tasca-emanuele',
 'camminata_tasca-luisa',
 'camminata_tasca-luisa',
 'camminata_tasca-simone',
 'corsa-alberto',
 'corsa-anna',
 'corsa-luisa',
 'corsa_tasca-alberto',
 'corsa_tasca-luisa',
 'quotidiano-alberto',
 'quotidiano-luisa',
 'salti-alberto',
 'shake-alberto',
 'shake-anna',
 'shake-daniele',
 'shake-luisa',
 'shake-mery']

In [6]:
tipo=[dir.split("-")[0] for dir in tipo]
tipo

['camminata',
 'camminata',
 'camminata',
 'camminata',
 'camminata',
 'camminata',
 'camminata',
 'camminata',
 'camminata_tasca',
 'camminata_tasca',
 'camminata_tasca',
 'camminata_tasca',
 'camminata_tasca',
 'camminata_tasca',
 'camminata_tasca',
 'corsa',
 'corsa',
 'corsa',
 'corsa_tasca',
 'corsa_tasca',
 'quotidiano',
 'quotidiano',
 'salti',
 'shake',
 'shake',
 'shake',
 'shake',
 'shake']

In [7]:
p = 150 # numero osservazioni per intervallo
nomi_colonna=["user","azione"]
nomi_colonna.extend(["a"+str(i) for i in range(p)])
nomi_colonna
X=pd.DataFrame(columns=nomi_colonna)

for i in tqdm.tqdm(range(len(DIR))):
    data = pd.read_csv(DIR[i] + "/accelerometer.txt", names = ["user", "type", "t", "ax", "ay", "az"]) # lettura dati
    data["t"] = data["t"] - data["t"].iloc[0] # t0 = 0
    data = data[(data["t"] > 7000) & (data["t"] < (data["t"].max()-7000))] # tolti i primi e ultimi 7 secondi
    data.reset_index(drop=True, inplace=True) # ripristinati gli indici da 0 in avanti
    data["a"] = (pd.to_numeric(data["ax"])**2 + pd.to_numeric(data["ay"])**2 + pd.to_numeric(data["az"])**2)**0.5 # accelerazione in modulo
    nome = [data.user[j] for j in range(0,len(data)-p, p)] # intervalli di dt*100ms
    tipologia=[tipo[i]]*len(nome)
    righe=[[nome[j],tipologia[j]] for j in range(len(nome))]
    [righe[j].extend(list(data.a[j*p:(j+1)*p])) for j in range(len(nome))]
    X=pd.concat([X,pd.DataFrame(righe,columns=nomi_colonna)],ignore_index=True) # ignore_index=T per avere indici consecutivi

100%|████████████████████████████████████████| 28/28 [00:03<00:00,  6.61it/s]


In [8]:
y = X.azione
X.drop("azione", axis=1, inplace=True)
Xnum = X.drop("user", axis=1)

In [9]:
# Funzioni per calcolare variazione media della derivata e sbiancare i dati
from funzioni import AbsMeanVarDeriv, Whiten

In [10]:
intTrapz = pd.DataFrame(np.trapz(Xnum, dx=10)) # integrale con approssimazione per trapezi
maxA = Xnum.max(1) # massimo accelerazione
MVDeriv = AbsMeanVarDeriv(Xnum, 10) # variazione media della derivata
Mean = Xnum.mean(axis=1)

In [11]:
# Costruzione del dataframe delle esplicative
espl = pd.concat([intTrapz, maxA, MVDeriv, Mean], axis=1)
espl.columns=["intTrapz", "maxA", "MVDeriv", "meanA"]

In [12]:
X.to_pickle("X-2s.pkl")
y.to_pickle("y-2s.pkl")
espl.to_pickle("espl.pkl")

In [13]:
espl

,intTrapz,maxA,MVDeriv,meanA
0,14398.936269,17.920838,0.001170,9.661280
1,15477.238332,19.667330,0.001723,10.383013
2,15260.431960,19.205382,0.001222,10.253074
3,14983.215855,19.026058,0.001365,10.073228
4,15871.838644,17.863954,0.001457,10.640028
5,14459.808679,19.894349,0.001381,9.712217
6,15811.753665,17.777913,0.001494,10.607282
7,14289.137038,20.315771,0.001095,9.606762
8,14951.162159,18.496067,0.001080,10.048357
9,15764.078516,18.190514,0.001400,10.586075
